# Harmony and STAC: PI 20.3 Demo

In PI 20.3, Harmony updated their service integrator's guide with improved guidance on transformation metadata to include in backend service outputs. In addition, we helped to ensure that the Harmony STAC catalog contains accurate spatial and temporal metadata representing the subsetted outputs produced by each backend service. 

This notebook provides a basic workflow to access Harmony outputs in-place from the s3 locations provided by the STAC catalog generated from an asynchronous request. For more a general introduction and tutorial, see [Harmony API Introduction](./Harmony%20Api%20Introduction.ipynb).  Useful helpers for making the calls found in this notebook can be found under the [docs/notebook-helpers](./notebook-helpers) folder.

## Prerequisites

#### You must run this notebook within an EC2 instance running in us-west-2:
1. Follow tutorials 01 through 03 of the [NASA Earthdata Cloud Primer](https://earthdata.nasa.gov/learn/user-resources/webinars-and-tutorials/cloud-primer) to set up an EC2 instance within us-west-2. Ensure you are also following step 3 in the ["Jupyter Notebooks on AWS EC2 in 12 (mostly easy) steps"](https://medium.com/@alexjsanchez/python-3-notebooks-on-aws-ec2-in-15-mostly-easy-steps-2ec5e662c6c6) article to set the correct security group settings needed to connect your local port to your EC2’s notebook port thru SSH.

2. Follow the remaining instructions in the Medium article above, which includes installation of Anaconda3 (including Jupyter Lab) in your ec2 instance. Before moving over to Jupyter Lab, perform steps 3 - 4 to set up Earthdata Login and Harmony access:

3. Setup your `~/.netrc` for Earthdata Login in your ec2 instance:

`machine uat.urs.earthdata.nasa.gov login <user> password <password>`

4. Run the following in your ec2 instance terminal window to generate short-term Harmony access keys:

`curl -Ln -bj https://harmony.uat.earthdata.nasa.gov/cloud-access.sh`

5. Set your environment variables based on the keys provided in step 4:

`export AWS_ACCESS_KEY_ID='...
export AWS_SECRET_ACCESS_KEY='...'
export AWS_SESSION_TOKEN='...'
export AWS_DEFAULT_REGION='us-west-2'`

6. Once the notebook is running in Jupyter Lab, run the following cell to install Python dependencies, import necessary modules, and set notebook defaults:

In [ ]:
%load_ext autoreload
%autoreload

import sys
# Install dependencies into the Jupyter Kernel
!{sys.executable} -m pip install -q -r notebook_helpers/requirements.txt
!{sys.executable} -m pip install intake-stac
%matplotlib inline

# Import libraries used throughout the notebook
from notebook_helpers import get, post, show, get_data_urls, show_async, show_async_condensed, print_async_status, show_shape
import json
import intake

## ASF Data Transformations

The ASF gdal service provides subsetting, reformatting, and reprojection capabilities for ASF's Sentinel-1 Interferograms (BETA) product:

In [ ]:
asf_collection = 'C1225776654-ASF'
coverages_root = 'https://harmony.uat.earthdata.nasa.gov/{collection}/ogc-api-coverages/1.0.0/collections/{variable}/coverage/rangeset'

### Variable and spatial subsetting with projecting, reformtatting output to PNG and spatial constraints
Each parent NetCDF is approx. 60 MB and the subsetted pngs and geotiffs are well under 1 MB each.

In [ ]:
response = get(
    coverages_root.format(
        collection=asf_collection, 
        variable='science%2Fgrids%2Fdata%2Fcoherence'),
    params={
        'format': 'image/png',
        'outputcrs': 'EPSG:2230',
        'subset': [
            'lon(-115.5:-115.25)', 
            'lat(33:33.1)',
            'time("2020-03-13T00:00:00Z":"2020-03-13T23:59:59Z")'
            ]})

show_async(response)

## Explore the STAC response using `intake-stac`

Each asynchronous request response includes a [STAC](https://stacspec.org/) catalog that contains spatial and temporal metadata for each output, or STAC item. These metadata fields now reflect the values of the subsetted outputs themselves, providing transformation metadata for users. The [Pangeo gallery](http://gallery.pangeo.io/repos/pangeo-data/pangeo-tutorial-gallery/intake.html) includes great guidance on how to work with stac catalogs to access cloud-hosted data in place.

#### Store job ID to create STAC location

In [ ]:
results = json.loads(response.content)
job = results['jobID']
print(job)

stac_root = 'https://harmony.uat.earthdata.nasa.gov/stac/{jobID}/{item}'

#### Open STAC Catalog from Harmony async response

Two STAC items are listed, corresponding to the two outputs plotted above.

In [ ]:
stac_cat = intake.open_stac_catalog(stac_root.format(jobID=job,item=''),name='Harmony output')
display(list(stac_cat))

In [ ]:
list(stac_cat)[0]

We can inspect the metadata of each STAC item, which includes the bounding box, coordinates, and start and end time:

In [ ]:
for i in range(len(list(stac_cat))):
    display(intake.open_stac_item(stac_root.format(jobID=job,item=i)))

Each item can be accessed from the harmony s3 staging bucket:

In [ ]:
entries = []
for id, entry in stac_cat.search('type').items():
    display(entry)
    entries.append(entry)

## Access Harmony outputs directly from STAC 

The Harmony output image is loaded up into an xarray data array directly from the STAC catalog.

In [ ]:
da = stac_cat[list(stac_cat)[0]][entries[0].describe()['name']].to_dask()
da

In [ ]:
da.plot.imshow()

#### Compare to non subsetted granule

The STAC metadata reflect the native granule bounds for an equivalent request without spatial subsetting:

In [ ]:
response_nosubset = get(
    coverages_root.format(
        collection=asf_collection, 
        variable='science%2Fgrids%2Fdata%2Fcoherence'),
    params={
        'format': 'image/png',
        'granuleID': 'G1234646236-ASF',
        'outputcrs': 'EPSG:2230',
        'forceAsync' : 'true',
        'subset': [
            'time("2020-03-13T00:00:00Z":"2020-03-13T23:59:59Z")'
            ]})
show_async_condensed(response_nosubset)

In [ ]:
results_nosubset = json.loads(response_nosubset.content)
job_nosubset = results_nosubset['jobID']

stac_cat_nosubset = intake.open_stac_catalog(stac_root.format(jobID=job_nosubset,item=''),name='Harmony output')

display(intake.open_stac_item(stac_root.format(jobID=job_nosubset,item='0')))